In [ ]:
import numpy as np
import itertools
from collections import defaultdict
from collections import Counter
from collections import namedtuple

In [ ]:
with open("day19_2.txt", "r") as fid:
    dat = fid.read()

In [ ]:
scanner = []
data = []
for line in dat.split("\n"):
    if line.startswith("---"):
        if len(data) > 0:
            scanner.append(np.array(data))
        data = []
    elif len(line) > 0 :
        data.append(list(map(int, line.split(","))))
scanner.append(np.array(data))

In [ ]:
r = [f"{x:03b}" for x in range(8)]
s = [[1 if x=='0' else -1 for x in rr]  for rr in r]

In [ ]:
m = np.zeros((8,3,3))

In [ ]:
for idx, ss in enumerate(s):
    m[idx] = np.diag(ss)

In [ ]:
mm = [] 
for j in list(itertools.permutations([0,1,2])):
    mm.append(m[:,j])
m = np.array(mm).reshape((-1,3,3))

In [ ]:
def comp(t0 , t1):
    m_idx = -1
    m0 , m1= t0.shape[0], t1.shape[0]
    res = []
    for t in t0:
        x = (t - np.matmul(t1, m) )
        res.append(x)

    rrr = np.moveaxis(np.array(res), (1),(0)).reshape((-1,m0 * m1, 3)) 

    for idx, r in enumerate(rrr):
        cnt = Counter()
        bv = [vec(*bi) for bi in r]
        cnt.update(bv)
        if cnt.most_common(1)[0][1] >= 12:
            m_idx = idx
            cord = cnt.most_common(1)[0][0]
            
    if(m_idx >= 0):
        cord_m = m[m_idx]
        func = lambda x: np.matmul(x, cord_m) + cord
        ifunc = lambda x: np.matmul(x, np.linalg.inv(cord_m)) -  np.matmul(np.linalg.inv(cord_m) , np.array(cord))
        return  (m_idx, func, ifunc)
    return -1 , 0
    

In [ ]:
maping = defaultdict(int)
graph = np.zeros((len(scanner), len(scanner)))

In [ ]:
vec = namedtuple('Point', ['x', 'y','z'])

In [ ]:
maping = defaultdict(int)
graph = np.zeros((len(scanner), len(scanner)))
for i in range(len(scanner)+1):
    for j in range(i + 1, len(scanner)):
        ret = comp(scanner[i], scanner[j]) 
        if ret[0] >=0:
            graph[i, j] = 1
            graph[j, i] = 1
            maping[f"{j}_{i}"] = ret[1]
            maping[f"{i}_{j}"] = ret[2]

In [ ]:
def findpath(path):
    npath = []
    for p in path:
        #print(f"p {p}")
        nnode = np.where(graph[p[-1]])[0]
        #print(f"n {nnode}")
        for n in nnode:
            if n not in p:
                npath.append(p.copy() + [n])
        #        print(f"new {p.copy() + [n]}")
    return npath

In [ ]:
def findtraj(cur, graph):
    npath = [[cur]]
    path = []
    while npath != path:
        #print(path)
        path = npath
        npath = findpath(path)
        for p in path:
            if p[-1] == 0:
                return p
                break

In [ ]:
allsets = {vec(*t) for t in scanner[0]}
for i in range(1, len(scanner)):
    #print(f"{i}:")
    tra = findtraj(i, graph)
    x = scanner[i].copy()
    #print(f"path {tra}")
    for s, d in zip(tra[:-1] , tra[1:]):
        #print(f"{s}_{d}")
        x = maping[f"{s}_{d}"](x)
    n = {vec(*t) for t in x}
    allsets = allsets.union(n)
    print(len(allsets))